In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
#GPU
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
df=pd.read_csv("/content/drive/MyDrive/sentiment/floread/sentiment-analysis/data/병합데이터셋.csv")
df.head()

,Unnamed: 0,sentence,emotion
0,0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,분노
1,1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,분노
2,2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,분노


In [6]:
#감정을 정수 라벨로 변경
emotions = {'행복': 0, '불안': 1, '놀람': 2, '슬픔': 3, '분노': 4, '중립': 5 }
df['emotion'] = df.emotion.map(emotions)
df.sample(n=5)

,Unnamed: 0,sentence,emotion
119791,31131,너. 결혼정보회사 가입할려구 나랑 헤어진다 그랬어?,5
26400,26400,소중한 예쁜 아기를 건강하게 출산했어. 정말 감사한 일인 것 같아.,0
105974,16082,사람 많이 변했네...,5
56131,4531,노후준비를 해둔 게 없는데 이제 은퇴를 해야 해서 두려워.,1
89066,30887,길가에 버려진 우리를 당신이 다시 채워주고 있습니다.,0


In [7]:
train_data=df.sample(frac=0.8, random_state=7)
test_data=df.drop(train_data.index)

In [8]:
MODEL_NAME="beomi/KcELECTRA-base"
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)

In [9]:
tokenized_train_sentences=tokenizer(
    list(train_data["sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [10]:
print(tokenized_train_sentences[0])
print(tokenized_train_sentences[0].tokens)
print(tokenized_train_sentences[0].ids)
print(tokenized_train_sentences[0].attention_mask)

Encoding(num_tokens=121, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['ĠìĽĲëŀĺ', 'ĠëĮĢíĳľëĭĺ', 'ĠìĬ¤íĥĢ', 'ìĿ¼ìĿ´', 'Ġê·¸ëłĩ', 'ìĬµëĭĪëĭ¤', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

In [11]:
tokenized_test_sentences=tokenizer(
    list(test_data["sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [12]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels):
    self.encodings=encodings
    self.labels=labels
  
  def __getitem__(self,idx):
    #item={key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item={key: val[idx].clone().detach() for key, val in self.encodings.items()}
    item["labels"]=torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [13]:
train_label=train_data["emotion"].values
test_label=test_data["emotion"].values

train_dataset=CurseDataset(tokenized_train_sentences,train_label)
test_dataset=CurseDataset(tokenized_test_sentences,test_label)

In [14]:
model=AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels=6)
model.to(device)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.weight', 'classifier.

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [15]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate

In [16]:
training_args=TrainingArguments(
    output_dir='./',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=500,
    save_total_limit=2
)

In [17]:
def compute_metrics(pred):
  labels=pred.label_ids
  preds=pred.predictions.argmax(-1)
  precision,recall,f1,_=precision_recall_fscore_support(labels,preds,average='weighted')
  acc=accuracy_score(labels,preds)
  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [18]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [19]:
#def change_transformers_dataset_2_right_format(dataset, label_name):
#  return dataset.map(lambda example: {'label': example[label_name]}, remove_columns=[label_name])

In [20]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.412500
1000,1.276400
1500,1.184700
2000,1.111000
2500,1.050600
3000,1.029300
3500,1.038500
4000,1.025700
4500,1.009100
5000,1.017200


TrainOutput(global_step=71170, training_loss=0.641989376673257, metrics={'train_runtime': 14384.5677, 'train_samples_per_second': 39.579, 'train_steps_per_second': 4.948, 'total_flos': 3.540251960729748e+16, 'train_loss': 0.641989376673257, 'epoch': 5.0})

In [21]:
trainer.evaluate(eval_dataset=test_dataset)

{'eval_loss': 1.627020001411438,
 'eval_accuracy': 0.6809990515333544,
 'eval_f1': 0.6793564608548797,
 'eval_precision': 0.6784231122201405,
 'eval_recall': 0.6809990515333544,
 'eval_runtime': 189.8099,
 'eval_samples_per_second': 149.976,
 'eval_steps_per_second': 2.344,
 'epoch': 5.0}

In [23]:
torch.save(model,'/content/drive/MyDrive/sentiment/floread/sentiment-analysis/model/kcelectra-v2.pt')

In [24]:
# 모델 사이즈 확인(파라미터는 v1과 동일)
import os

model_path = '/content/drive/MyDrive/sentiment/floread/sentiment-analysis/model/kcelectra-v2.pt'
size2 = os.path.getsize(model_path) / (1024*1024) # mb 단위
print(f"Model size: {size2:.2f} MB")

Model size: 416.25 MB
